In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch
import trimesh
from functools import partial
import ssl
import pickle
from Solvers import SolverEmbedding,Loss
from Models.EncoderModels import TextEncoder, ShapeEncoder
from config import cfg
from dataEmbedding.dataEmbedding import Read_Load_BuildBatch
from dataEmbedding.dataEmbeddingLoader import GenerateDataLoader,check_dataset,collate_embedding
from dataEmbedding.generateEmbedding import build_embeedings_CWGAN
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device=torch.device("cuda")
else:
    device=torch.device("cpu")

device=cfg.DEVICE
print(device)
PYTORCH_ENABLE_MPS_FALLBACK=1
#for mac os fix 
ssl._create_default_https_context = ssl._create_unverified_context

cpu


In [2]:
stanData=Read_Load_BuildBatch(cfg.EMBEDDING_VOXEL_FOLDER,cfg.EMBEDDING_BATCH_SIZE)


In [3]:
stanData.wordlens.sort()
mid = len(stanData.wordlens) // 2
res = (stanData.wordlens[mid] + stanData.wordlens[~mid]) / 2
print("Median of list is : " + str(res))
print("Mean: "+ str(sum(stanData.wordlens)/len(stanData.wordlens)))

Median of list is : 14.0
Mean: 16.307937873199133


In [4]:
criterion={
        'walker': Loss.RoundTripLoss(device=device),
        'visit': Loss.AssociationLoss(device=device),
        #'metric': Loss.SmoothedMetricLoss(device=device)
        #'metric': Loss.InstanceMetricLoss()
        #'metric': Loss.TripletLoss()
        'metric':Loss.NPairLoss()
        }

In [5]:
ShapeModel=ShapeEncoder()
ShapeModel=ShapeModel.to(device)
TextModel=TextEncoder(len(stanData.dict_word2idx))
TextModel=TextModel.to(device)
optimizer = torch.optim.Adam(list(ShapeModel.parameters()) + list(TextModel.parameters()), lr=cfg.EMBEDDING_LR, weight_decay=cfg.EMBEDDING_WEIGHT_DC)
history=SolverEmbedding.Solver(TextModel,ShapeModel,stanData,optimizer,criterion,cfg.EMBEDDING_BATCH_SIZE,'online',device)

In [6]:
shape1 = (64, 128)
shape2 = (64, 128)

# Create random matrices with the specified shapes
matrix1 = torch.randn(shape1)
matrix2 = torch.randn(shape2)

result_vector = (matrix1 @ matrix2.transpose(0,1)).diagonal()

# Transpose the second matrix along the first two dimensions (64, 128, 63)
#transposed_matrix2 = matrix2.transpose(1, 2)

# Initialize an empty result matrix
#result = torch.zeros((shape1[0], shape2[1]))

# Perform the element-wise multiplications and summation
#for i in range(shape1[0]):
#    result[i] = torch.matmul(matrix1[i],transposed_matrix2[i]) -3

# Print the shape of the result
#print(result.shape)

RuntimeError: The size of tensor a (128) must match the size of tensor b (64) at non-singleton dimension 1

In [7]:
history.train(cfg.EMBEDDING_EPOCH_NR,stanData.dict_idx2word)

Epoch [1/4] starting...

Training...


0it [00:00, ?it/s]

tensor([[  3,   5,   7,  ..., 123, 125, 127],
        [  1,   5,   7,  ..., 123, 125, 127],
        [  1,   3,   7,  ..., 123, 125, 127],
        ...,
        [  1,   3,   5,  ..., 121, 125, 127],
        [  1,   3,   5,  ..., 121, 123, 127],
        [  1,   3,   5,  ..., 121, 123, 125]])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 63, 128])


RuntimeError: The size of tensor a (128) must match the size of tensor b (64) at non-singleton dimension 1

In [21]:
loader={'train' : GenerateDataLoader(stanData.train,stanData.data_dir,stanData.dict_word2idx),
        'test' : GenerateDataLoader(stanData.test,stanData.data_dir,stanData.dict_word2idx),
        'val':GenerateDataLoader(stanData.val,stanData.data_dir,stanData.dict_word2idx)}

dataloader = {
            'train': DataLoader(
                loader['train'], 
                batch_size=cfg.EMBEDDING_BATCH_SIZE * 2,              
                drop_last=check_dataset(loader['train'], cfg.EMBEDDING_BATCH_SIZE * 2),
                collate_fn=collate_embedding,
                num_workers=4
            ),
            'val': DataLoader(
                loader['val'], 
                batch_size=cfg.EMBEDDING_BATCH_SIZE*2,
                collate_fn=collate_embedding,
                num_workers=4
            ),
            'test': DataLoader(
                loader['test'], 
                batch_size=cfg.EMBEDDING_BATCH_SIZE*2,
                collate_fn=collate_embedding
                #num_workers=2
            )
    }   
build_embeedings_CWGAN(cfg.EMBEDDING_TEXT_MODELS_PATH,TextEncoder(len(stanData.dict_word2idx)),dataloader,cfg.EMBEDDING_SAVE_PATH,cfg.DEVICE)


100%|██████████| 30/30 [00:27<00:00,  1.08it/s]
